In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GRU, Bidirectional, GlobalAveragePooling2D, Reshape
from tensorflow.keras.metrics import Precision, Recall
import matplotlib.pyplot as plt

In [2]:
# Custom F1Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to binary labels (threshold at 0.5)
        y_pred = tf.cast(y_pred >= 0.5, tf.float32)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [3]:
# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Set paths for your dataset
train_directory = r'/Music/Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/'  # Update with the correct path for training images
test_directory = r'Music/Monkeypox/Kaggle MP images/archive (2)/Augmented Images/Augmented Images/'  # Update with the correct path for testing images

In [5]:
# Image data generators for training and testing
training_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    test_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 3192 images belonging to 2 classes.
Found 3192 images belonging to 2 classes.


In [6]:
# Model Definition with CNN + Bi-GRU
model = Sequential()

In [7]:
# Add Conv2D layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Add Global Average Pooling
model.add(GlobalAveragePooling2D())

In [9]:
# Reshape the output of the Conv2D layers to make it suitable for GRU
model.add(Reshape((1, 128)))  # Reshape to (batch_size, time_steps, features)

In [10]:
# Add Bi-GRU (Bidirectional GRU) layer
model.add(Bidirectional(GRU(64, return_sequences=False, activation='relu')))

In [11]:
# Add Dropout and Dense layers for classification
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

In [12]:
# Compile the model with custom F1 score and other metrics
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall(), F1Score()]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 167,873 (655.75 KB)

 Trainable params: 167,873 (655.75 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=100
)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 127s 712ms/step - accuracy: 0.5782 - f1_score: 0.7042 - loss: 0.6745 - precision: 0.5842 - recall: 0.8880 - val_accuracy: 0.6623 - val_f1_score: 0.7444 - val_loss: 0.6255 - val_precision: 0.6398 - val_recall: 0.8900
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 603ms/step - accuracy: 0.6474 - f1_score: 0.7283 - loss: 0.6362 - precision: 0.6397 - recall: 0.8458 - val_accuracy: 0.6726 - val_f1_score: 0.7293 - val_loss: 0.6150 - val_precision: 0.6714 - val_recall: 0.7982
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 712ms/step - accuracy: 0.6535 - f1_score: 0.7230 - loss: 0.6350 - precision: 0.6445 - recall: 0.8236 - val_accuracy: 0.6717 - val_f1_score: 0.7217 - val_loss: 0.6119 - val_precision: 0.6788 - val_recall: 0.7704
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 82s 664ms/step - accuracy: 0.6778 - f1_score: 0.7388 - loss: 0.6097 - precision: 0.6689 - recall: 0.8255 - val_accuracy: 0.6942 - val_f1_score: 0.7583 - val_loss: 0.5935 - val_precision: 0.6

In [14]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot_cnn_bgru.png')  # Save the plot as a PNG file
plt.close()

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot_cnn_bgru.png')  # Save the plot as a PNG file
plt.close()

# Save Precision plot
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot_cnn_bgru.png')  # Save the plot as a PNG file
plt.close()

# Save Recall plot
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot_cnn_bgru.png')  # Save the plot as a PNG file
plt.close()

# Save F1-Score plot
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot_cnn_bgru.png')  # Save the plot as a PNG file
plt.close()